In [ ]:
# Install required packages
!pip install anthropic


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.5/865.5 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement you-search (from versions: none)
ERROR: No matching distribution found for you-search


In [ ]:
pip install knowledge-storm

  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nv

In [ ]:
# Import necessary libraries
import os
from argparse import ArgumentParser

from knowledge_storm import STORMWikiRunnerArguments, STORMWikiRunner, STORMWikiLMConfigs
from knowledge_storm.lm import ClaudeModel
from knowledge_storm.rm import YouRM, BingSearch
from knowledge_storm.utils import load_api_key

# Set up environment variables
os.environ["ANTHROPIC_API_KEY"] = "your_anthropic_key_here"
os.environ["YDC_API_KEY"] = "your_YDC_api_key_here"



In [ ]:
import os
from dataclasses import dataclass

from knowledge_storm import STORMWikiRunnerArguments, STORMWikiRunner, STORMWikiLMConfigs
from knowledge_storm.lm import ClaudeModel
from knowledge_storm.rm import YouRM, BingSearch
from knowledge_storm.utils import load_api_key


@dataclass
class Args:
    output_dir: str = './results/claude'
    max_thread_num: int = 3
    retriever: str = 'you'  # or 'bing'
    do_research: bool = True
    do_generate_outline: bool = True
    do_generate_article: bool = True
    do_polish_article: bool = True
    max_conv_turn: int = 3
    max_perspective: int = 3
    search_top_k: int = 3
    retrieve_top_k: int = 3
    remove_duplicate: bool = False

def main(args):
    #load_api_key(toml_file_path='secrets.toml')
    lm_configs = STORMWikiLMConfigs()
    claude_kwargs = {
        'api_key': os.getenv("ANTHROPIC_API_KEY"),
        'temperature': 1.0,
        'top_p': 0.9
    }

    conv_simulator_lm = ClaudeModel(model='claude-3-haiku-20240307', max_tokens=500, **claude_kwargs)
    question_asker_lm = ClaudeModel(model='claude-3-sonnet-20240229', max_tokens=500, **claude_kwargs)
    outline_gen_lm = ClaudeModel(model='claude-3-opus-20240229', max_tokens=400, **claude_kwargs)
    article_gen_lm = ClaudeModel(model='claude-3-5-sonnet-20240620', max_tokens=700, **claude_kwargs)
    article_polish_lm = ClaudeModel(model='claude-3-5-sonnet-20240620', max_tokens=4000, **claude_kwargs)

    lm_configs.set_conv_simulator_lm(conv_simulator_lm)
    lm_configs.set_question_asker_lm(question_asker_lm)
    lm_configs.set_outline_gen_lm(outline_gen_lm)
    lm_configs.set_article_gen_lm(article_gen_lm)
    lm_configs.set_article_polish_lm(article_polish_lm)

    engine_args = STORMWikiRunnerArguments(
        output_dir=args.output_dir,
        max_conv_turn=args.max_conv_turn,
        max_perspective=args.max_perspective,
        search_top_k=args.search_top_k,
        max_thread_num=args.max_thread_num,
    )

    if args.retriever == 'bing':
        rm = BingSearch(bing_search_api=os.getenv('BING_SEARCH_API_KEY'), k=engine_args.search_top_k)
    elif args.retriever == 'you':
        rm = YouRM(ydc_api_key=os.getenv('YDC_API_KEY'), k=engine_args.search_top_k)

    runner = STORMWikiRunner(engine_args, lm_configs, rm)

    topic = input('Topic: ')
    runner.run(
        topic=topic,
        do_research=args.do_research,
        do_generate_outline=args.do_generate_outline,
        do_generate_article=args.do_generate_article,
        do_polish_article=args.do_polish_article,
    )
    runner.post_run()
    runner.summary()

# Create an instance of Args with default values
args = Args()

# You can modify any of the arguments here if needed
# For example:
# args.output_dir = '/content/my_output'
# args.retriever = 'bing'

# Run the main function
main(args)

Topic: Swami Vivekananda on work and its secret


ERROR:root:Error occurs when searching query : 'hits'
ERROR:root:Error occurs when searching query : 'hits'


***** Execution time *****
run_knowledge_curation_module: 85.8875 seconds
run_outline_generation_module: 30.0071 seconds
run_article_generation_module: 29.4752 seconds
run_article_polishing_module: 7.5474 seconds
***** Token usage of language models: *****
run_knowledge_curation_module
    claude-3-haiku-20240307: {'prompt_tokens': 12760, 'completion_tokens': 5931}
    claude-3-sonnet-20240229: {'prompt_tokens': 10348, 'completion_tokens': 3133}
    claude-3-opus-20240229: {'prompt_tokens': 0, 'completion_tokens': 0}
    claude-3-5-sonnet-20240620: {'prompt_tokens': 0, 'completion_tokens': 0}
run_outline_generation_module
    claude-3-haiku-20240307: {'prompt_tokens': 0, 'completion_tokens': 0}
    claude-3-sonnet-20240229: {'prompt_tokens': 0, 'completion_tokens': 0}
    claude-3-opus-20240229: {'prompt_tokens': 5118, 'completion_tokens': 580}
    claude-3-5-sonnet-20240620: {'prompt_tokens': 0, 'completion_tokens': 0}
run_article_generation_module
    claude-3-haiku-20240307: {'promp